# Hybrid Approach
- Deep learning to localize the sign using YOLO
- classical ML to recognize the traffic sign using SVM

# Dataset Manipulation

### dataset to YOLO format

In [1]:
import os
import shutil

# Paths
root_dir = "YOLODataset"
ts_dir = os.path.join(root_dir, "ts")
train_txt = os.path.join(root_dir, "train.txt")
val_txt = os.path.join(root_dir, "test.txt")

# Output YOLO structure
output_dir = "YOLO_dataset"
images_train = os.path.join(output_dir, "images/train")
images_val = os.path.join(output_dir, "images/val")
labels_train = os.path.join(output_dir, "labels/train")
labels_val = os.path.join(output_dir, "labels/val")

# Create directories
os.makedirs(images_train, exist_ok=True)
os.makedirs(images_val, exist_ok=True)
os.makedirs(labels_train, exist_ok=True)
os.makedirs(labels_val, exist_ok=True)

def process_split(txt_path, image_dst, label_dst):
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        image_path = line.strip()
        image_name = os.path.basename(image_path)
        label_name = os.path.splitext(image_name)[0] + ".txt"

        full_image_path = os.path.join(ts_dir, image_name)
        full_label_path = os.path.join(ts_dir, label_name)

        # Copy image
        shutil.copy(full_image_path, os.path.join(image_dst, image_name))

        # Copy label
        if os.path.exists(full_label_path):
            shutil.copy(full_label_path, os.path.join(label_dst, label_name))
        else:
            print(f"⚠️ Label not found for {image_name}")

# Process train and val splits
process_split(train_txt, images_train, labels_train)
process_split(val_txt, images_val, labels_val)

# Create data.yaml
data_yaml_path = os.path.join(output_dir, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"""train: {os.path.abspath(images_train)}
val: {os.path.abspath(images_val)}

nc: 4
names: ['prohibitory', 'danger', 'mandatory', 'other']
""")

print("✅ Dataset prepared at:", output_dir)


✅ Dataset prepared at: YOLO_dataset


# Train YOLO detector

In [11]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [13]:
model.train(data="YOLO_dataset/data.yaml", epochs=5, imgsz=960, batch=16)

New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=YOLO_dataset/data.yaml, epochs=5, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, vis

## predict

In [1]:
from ultralytics import YOLO
import cv2

# Class names for your model (match your `data.yaml`)
CLASS_NAMES = ['prohibitory', 'danger', 'mandatory', 'other']

# Load trained model
model = YOLO("runs/detect/train8/weights/best.pt")

# Load image
img_path = "YOLO_dataset/images/val/00004.jpg"
image = cv2.imread(img_path)

# Run prediction
results = model.predict(source=img_path, conf=0.5)

# Get first result
boxes = results[0].boxes

# Draw bounding boxes
for box in boxes:
    # Get box coordinates (xyxy format: [x1, y1, x2, y2])
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    label = f"{CLASS_NAMES[cls_id]} {conf:.2f}"

    print(f"Class: {label}, BBox: {x1, y1, x2, y2}")

    # Draw box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Show image
cv2.imshow("Detected Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


c:\Users\OSAMA\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

image 1/1 c:\osama\CUFE\5-Senior2\GP\ADAS-System-prototype\Traffic_Recognition\YOLO_dataset\images\val\00004.jpg: 576x960 1 prohibitory, 1 danger, 230.7ms
Speed: 9.6ms preprocess, 230.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 960)


Class: danger 0.99, BBox: (897, 342, 971, 408)
Class: prohibitory 0.87, BBox: (905, 406, 956, 458)
